In [1]:
"""
读取数据
"""
import pandas as pd
df = pd.read_csv("data/news.csv", encoding="utf-8")
texts = df.iloc[:, 0].astype(str).tolist()
print(f"成功读取 {len(texts)} 条文本数据。")

成功读取 1999 条文本数据。


In [2]:
"""
数据清洗
"""
import re, string
cleaned_texts = []
for text in texts:
    text = re.sub(r"\s+", " ", text)
    punctuations = string.punctuation + "，。！？、；：“”‘’（）《》〈〉【】{}【】——…￥"
    text = re.sub(f"[{re.escape(punctuations)}]", "", text)
    text = text.lower()
    cleaned_texts.append(text)

In [3]:
"""
分词
"""
import jieba
import jieba.posseg as pseg

# 加载自定义词典
jieba.load_userdict("data/newdic1.txt")

tokenized_texts = [list(jieba.cut(text)) for text in texts]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\32087\AppData\Local\Temp\jieba.cache
Loading model cost 0.473 seconds.
Prefix dict has been built successfully.


In [4]:
"""
去除停用词
"""
with open("data/stopword.txt", "r", encoding="utf-8") as f:
    stopwords = set(line.strip() for line in f)
print(f"加载停用词表，共 {len(stopwords)} 个停用词。")

filtered_texts = []
for words in tokenized_texts:
    filtered = [word for word in words if word not in stopwords and word.strip()]
    filtered_texts.append(filtered)

加载停用词表，共 1895 个停用词。


In [5]:
"""
去重
"""
df["processed_text"] = [" ".join(words) for words in texts]
before_dedup = len(df)
dedup_df = df.drop_duplicates(subset=["processed_text"])
after_dedup = len(dedup_df)
print(f"去重完成，去除了 {before_dedup - after_dedup} 条重复记录。")
dedup_texts = dedup_df["processed_text"].apply(lambda x: x.split()).tolist()

去重完成，去除了 90 条重复记录。


In [6]:
"""
去除低频词
"""
from collections import Counter
counter = Counter(word for words in filtered_texts for word in words)
print(f"词汇总数: {len(counter)}")

# 筛选高频词
filtered = [
    [word for word in words if counter[word] >= 5]
    for words in filtered_texts
]
print(f"移除词频低于 5 的词汇。")

# 更新词频统计
updated_counter = Counter(word for words in filtered for word in words)

词汇总数: 78920
移除词频低于 5 的词汇。


In [7]:
"""
词性标注
"""
pos_tagged_texts = []
for words in filtered:
    words_joined = " ".join(words)
    words_pseg = pseg.cut(words_joined)
    pos_tags = [(word, flag) for word, flag in words_pseg]
    pos_tagged_texts.append(pos_tags)

In [8]:
"""
命名实体识别
"""
import spacy

nlp = spacy.load("zh_core_web_sm")
ner_results = []
for words in filtered:
    text = "".join(words)
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    ner_results.append(entities)

In [9]:
"""
词频统计
"""
for word, freq in updated_counter.most_common(20):
    print(f"{word}: {freq}")

中: 4157
年: 3769
孩子: 2865
学校: 2505
学生: 2420
旅游: 2282
中国: 2248
说: 2085
时: 1855
招生: 1841
分: 1573
教育: 1531
时间: 1468
10: 1458
广州: 1443
考生: 1398
游客: 1382
记者: 1336
录取: 1217
广东: 1195


In [10]:
"""
保存处理后的结果到文件
"""
with open("output/processed_news.txt", "w", encoding="utf-8") as f:
    for i in range(len(filtered_texts)):
        words = filtered_texts[i]
        pos_tags = pos_tagged_texts[i]
        entities = ner_results[i]
        f.write(f"文本 {i+1}:\n")
        f.write("分词: " + " ".join(words) + "\n")
        f.write(
            "词性标注: "
            + " ".join([f"{word}/{flag}" for word, flag in pos_tags])
            + "\n"
        )
        f.write(
            "实体识别: "
            + ", ".join([f"{ent[0]}/{ent[1]}" for ent in entities])
            + "\n"
        )
        f.write("\n")

In [11]:
"""
保存词频统计结果
"""
with open("output/word_frequency.txt", "w", encoding="utf-8") as f:
    for word, freq in counter.most_common():
        f.write(f"{word}: {freq}\n")